In [ ]:
!pip install langchain
!pip install torch
!pip install anaconda
!pip install transformes
!pip install jupyter
!pip install ipywidgets widgetsnbextension pandas-profiling
!pip install accelerate 
!pip install ctransformers
!pip install 'transformers[torch]'
!pip install 'transformers[tf-cpu]'
!pip install pypdf
!pip install pdf2image
!pip install pdfminer
!pip install unstructured
!pip3 install pdfminer.six
!pip install sentence-transformers
!pip install chromadb

In [8]:
from langchain import PromptTemplate

template = """<|prompter|>{question}<|endoftext|><|assistant|>"""

prompt = PromptTemplate(template=template, input_variables=["question"])

In [34]:
from langchain_openai import ChatOpenAI
import a_env_vars
import os
os.environ["OPENAI_API_KEY"] = a_env_vars.OPENAI_API_KEY

llm = ChatOpenAI(temperature=0,model_name='gpt-3.5-turbo')


In [32]:
from langchain import LLMChain

llm_chain = LLMChain(prompt=prompt, llm=llm)

question = "What is the meaning of life?"

llm_chain.run(question)

'The meaning of life is a deeply philosophical question that has been pondered by humans for centuries. Different people and cultures have different beliefs and interpretations of the meaning of life. Some believe that the meaning of life is to seek happiness and fulfillment, while others believe it is to fulfill a higher purpose or spiritual calling. Ultimately, the meaning of life is a personal and subjective concept that each individual must explore and define for themselves.'

In [35]:
from langchain.document_loaders import PyPDFLoader

loader = PyPDFLoader("c:\dev\contrato.pdf")
document = loader.load()

In [36]:
from langchain.text_splitter import CharacterTextSplitter

text_splitter = CharacterTextSplitter(chunk_size=1024, chunk_overlap=64)

# texts = text_splitter.split_text(raw_text)
documents = text_splitter.split_documents(document)

In [37]:
from langchain.embeddings import HuggingFaceEmbeddings

embeddings = HuggingFaceEmbeddings()

query_result = embeddings.embed_query(documents[0].page_content)

query_result

[0.0023064822889864445,
 0.0682450532913208,
 -0.026508817449212074,
 0.026515088975429535,
 -0.005142151843756437,
 -0.03132065758109093,
 0.031963229179382324,
 -0.032870396971702576,
 -0.03622788190841675,
 0.026239311322569847,
 -0.009984185919165611,
 0.04248514026403427,
 0.009718108922243118,
 0.09529905766248703,
 0.03282051533460617,
 -0.07111258059740067,
 0.00010368250514147803,
 -0.03442153334617615,
 -0.04651372507214546,
 0.012837479822337627,
 -0.06202609837055206,
 0.03988494351506233,
 -0.10192443430423737,
 0.018759042024612427,
 0.05363691970705986,
 -0.02336989901959896,
 -0.04968152567744255,
 -0.037421949207782745,
 0.02002786472439766,
 -0.02389957569539547,
 0.06198689341545105,
 0.013441342860460281,
 0.030533472076058388,
 -0.004157816991209984,
 2.4416656287939986e-06,
 -0.04435492306947708,
 -0.04466623067855835,
 0.012117289006710052,
 -0.027305105701088905,
 0.03588144853711128,
 -0.030993901193141937,
 0.03288203105330467,
 0.006711556576192379,
 0.020676

In [38]:
from langchain.vectorstores import Chroma

vectorstore = Chroma.from_documents(documents, embeddings)

In [39]:
from langchain.chains import ConversationalRetrievalChain

qa = ConversationalRetrievalChain.from_llm(llm, vectorstore.as_retriever(), return_source_documents=True)


In [46]:
chat_history = []
query = "armar una lista con la comisiones adicionales, con las siguientes entradas: origen, destino,clases,porcentaje"
result = qa({"question": query, "chat_history": chat_history})
result["answer"]

'Origen: Chile  \nDestino: Canadá, EE. UU., Europa y Asia-Pacífico  \nClases: J, C, D, Z, P - 8%  \nClases: O, E, A - 8%  \nClases: Y, B, M, U, H, Q, V, W, S, T, L, K, G* - 8%  \n*Las tarifas básicas para rutas entre Chile y Canadá/EE. UU. están excluidas de recibir comisión adicional. Las tarifas básicas se encuentran en las clases G, K y L y finalizarán su base tarifaria con -BA.'